In [18]:
# Импорт библиотек
import numpy as np  # для матричных вычислений
import pandas as pd  # для анализа и предобработки данных

from sklearn import linear_model  # линейные модели
from sklearn import ensemble  # ансамбли
from sklearn import metrics  # метрики
from sklearn.model_selection import train_test_split  # сплитование выборки
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV  
from hyperopt import hp, fmin, tpe, Trials
from sklearn.model_selection import cross_val_score
import optuna
 

In [19]:
data = pd.read_csv('data/_train_sem09 (1).csv')
display(data.shape)
data.head()

(3751, 1777)

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [20]:
data['Activity'].value_counts(normalize=True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

Данные сбалансированы, предобработка не требуется.

## <center>Подготовка данных

In [21]:
# Выделим целевой признак
y = data['Activity']
x = data.drop(['Activity'], axis=1)
# Разделим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state = 42, test_size = 0.2)

## <center>Обучение с параметрами "по умолчанию"

In [22]:
# Обучим логистическую регрессию и выведем метрики
log_reg = linear_model.LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)
y_train_log_reg = log_reg.predict(X_train)
y_test_log_reg = log_reg.predict(X_test)
print('Логистическая регрессия:')
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_log_reg)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_log_reg)), '\n')

# Обучим случайный лес с параметрами по умолчанию и выводим значения метрики 
rfc = ensemble.RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)
#Выводим значения метрики 
y_train_rfc = rfc.predict(X_train)
y_test_rfc = rfc.predict(X_test)
print('Случайный лес:')
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_rfc)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_rfc)))

Логистическая регрессия:
f1_score на тренировочном наборе: 0.89
f1_score на тестовом наборе: 0.78 

Случайный лес:
f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.80


## <center> Оптимизация гиперпараметров

###  GridSearchCV

Ручным перебором коэффициента *С* лучшие результаты метрики *F1* на тестовой выборке достигаются в диапазоне до 0.5, поэтому для экономии времени рассчёта с помощью GridSearchCV ограничим сетку этим значением.

In [23]:
# Зададим сетку гиперпараметров для логистической регрессии
param_grid_lr = [{'penalty': ['l1','l2'],  # тип регуляризации
    'solver': ['saga', 'liblinear'],  # алгоритм оптимизации
    'C': list(np.linspace(0.01, 0.5 , 6))},  # уровень силы регурялизации
    {'penalty': ['none'], 
    'solver': ['saga', 'lbfgs', 'sag', 'newton-cg']}]
    #{'penalty': ['l2', 'none'] ,
    #'solver': ['lbfgs', 'sag', 'newton-cg'],
    #'C': list(np.linspace(0.01, 1, 11))},
    #{'penalty': ['l1', 'l2'] ,
    #'solver': ['liblinear'],
    #'C': list(np.linspace(0.01, 1, 11))}

# Создадим объект GridSearch для логиситической регрессии
grid_search_lr = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid_lr, 
    cv=5, 
    n_jobs = -1
)  
# Подберём праметры и посчитаем метрики
grid_search_lr.fit(X_train, y_train) 
y_train_pred = grid_search_lr.predict(X_train)
y_test_pred = grid_search_lr.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

f1_score на тренировочном наборе: 0.83
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'C': 0.20600000000000002, 'penalty': 'l1', 'solver': 'saga'}


In [24]:
# Зададим сетку параметров для рандомного леса
param_grid_rfc = {'n_estimators': list(range(60, 240, 30)),
              'min_samples_leaf': list(range(1, 11, 1)),
              'max_depth': list(range(4, 22, 2))
              }
# Создадим объект GridSearch для рандомного леса
grid_search_rfc = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid_rfc, 
    cv=5, 
    n_jobs = -1
)  
# Подберём праметры и посчитаем метрики
grid_search_rfc.fit(X_train, y_train)
y_train_pred = grid_search_rfc.predict(X_train)
y_test_pred = grid_search_rfc.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rfc.best_params_))

f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 14, 'min_samples_leaf': 1, 'n_estimators': 180}


### **RandomizedSearchCV**

In [25]:
param_distr_lr = [{'penalty': ['l2', 'none'],  # тип регуляризации
    'solver': ['saga', 'lbfgs'],  # алгоритм оптимизации
    'C': list(np.linspace(0.01, 0.5 , 6))},  # уровень силы регурялизации
    {'penalty': ['l2', 'none'] ,
    'solver': ['lbfgs', 'sag', 'newton-cg'],
    'C': list(np.linspace(0.01, 1, 11))},
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear'],
    'C': list(np.linspace(0.01, 1, 11))}
    ]

random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distr_lr, 
    cv=5, 
    n_iter = 16, 
    n_jobs = -1
    )

random_search_lr.fit(X_train, y_train) 
y_train_pred = random_search_lr.predict(X_train)
y_test_pred = random_search_lr.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

f1_score на тренировочном наборе: 0.84
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'C': 0.20600000000000002, 'penalty': 'l1', 'solver': 'saga'}


In [26]:
param_distr_rfc = {'n_estimators': list(range(60, 240, 20)),
              'min_samples_leaf': list(range(1, 11, 1)),
              'max_depth': list(range(4, 22, 1))
              }

random_search_rfc = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distr_rfc, 
    cv=5, 
    n_iter = 22, 
    n_jobs = -1
    )

random_search_rfc.fit(X_train, y_train) 
y_train_pred = random_search_rfc.predict(X_train)
y_test_pred = random_search_rfc.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rfc.best_params_))

f1_score на тренировочном наборе: 1.00
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'max_depth': 14, 'min_samples_leaf': 1, 'n_estimators': 180}


### Hyperopt

#### Логистическая регрессия

In [27]:
# Зададим пространство поиска для логистической регрессии
solver_list = ['lbfgs', 'newton-cg', 'sag', 'saga']
penalty_list = ['l2']
space_lr={'solver': hp.choice('solver', solver_list),
       'penalty' : hp.choice('penalty', penalty_list),
       'C': hp.uniform('C', 0.01, 1)}

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'solver': str(params['solver']), 
              'penalty': str(params['penalty']), 
             'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, max_iter=1000, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
 
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# Подберём гиперпараметры для линейной регрессии
trials_lr = Trials() # используется для логирования результатов

best_lr=fmin(hyperopt_lr,  # функция для максимизации
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_lr, # логирование результатов
          rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
         )

print("Наилучшие значения гиперпараметров {}".format(best_lr))

100%|██████████| 20/20 [07:32<00:00, 22.61s/trial, best loss: -0.7902171664645037]
Наилучшие значения гиперпараметров {'C': 0.05084775379720359, 'penalty': 0, 'solver': 0}


In [28]:
# Обучим логистическую регрессию с параметрами полученными с помощью Hyperopt и посчитаем значение метрики на тестовой выборке
C = best_lr['C']
solver = solver_list[best_lr['solver']]
penalty = penalty_list[best_lr['penalty']]
log_reg = linear_model.LogisticRegression(max_iter=1000,
                                          random_state=42,
                                          C=C,
                                          solver=solver,
                                          penalty=penalty)
log_reg.fit(X_train, y_train)
y_train_log_reg = log_reg.predict(X_train)
y_test_log_reg = log_reg.predict(X_test)

print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_log_reg)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_log_reg)), '\n')


f1_score на тренировочном наборе: 0.84
f1_score на тестовом наборе: 0.79 



#### Случайный лес

In [29]:
# зададим пространство поиска гиперпараметров для случайного леса
space_rfc={'n_estimators': hp.quniform('n_estimators', 60, 270, 10),
       'max_depth' : hp.quniform('max_depth', 4, 24, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 11, 1)
      }

def hyperopt_rfc(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
 
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# Подберём гиперпараметры для линейной регрессии
trials_rfc = Trials()  # используется для логирования результатов

best_rfc=fmin(hyperopt_rfc,  # наша функция 
          space=space_rfc,  # пространство гиперпараметров
          algo=tpe.suggest,  # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=30,  # максимальное количество итераций
          trials=trials_rfc,  # логирование результатов
          rstate=np.random.default_rng(42)  # фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rfc))

100%|██████████| 30/30 [01:53<00:00,  3.77s/trial, best loss: -0.8185557076596979]
Наилучшие значения гиперпараметров {'max_depth': 14.0, 'min_samples_leaf': 1.0, 'n_estimators': 160.0}


In [30]:
# Обучим логистическую регрессию с параметрами полученными с помощью Hyperopt и посчитаем значение метрики на тестовой выборке
rfc = ensemble.RandomForestClassifier(n_estimators=int(best_rfc['n_estimators']),
                                      max_depth=int(best_rfc['max_depth']),
                                      min_samples_leaf=int(best_rfc['min_samples_leaf']),
                                      random_state=42)
rfc.fit(X_train, y_train)
#Выводим значения метрики 
y_train_rfc = rfc.predict(X_train)
y_test_rfc = rfc.predict(X_test)
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_rfc)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_rfc)))

f1_score на тренировочном наборе: 0.99
f1_score на тестовом наборе: 0.81


### Optuna

#### Логистическая регрессия

In [31]:
def optuna_lr(trial_optuna_lr):
  # задаем пространства поиска гиперпараметров
  solver = trial_optuna_lr.suggest_categorical('solver', solver_list)
  penalty = trial_optuna_lr.suggest_categorical('penalty', penalty_list)
  C = trial_optuna_lr.suggest_float('C', 0.01, 1)

  # создаем модель
  model = linear_model.LogisticRegression(solver=solver,
                                          penalty=penalty, 
                                          C=C,
                                          max_iter=1000,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

# cоздаем объект исследования
study_lr = optuna.create_study(study_name='LogisticRegression', direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr.optimize(optuna_lr, n_trials=16)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_lr.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_lr.best_value))\
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study_lr.best_params,random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-04-13 02:40:52,556] A new study created in memory with name: LogisticRegression
[I 2023-04-13 02:40:56,315] Trial 0 finished with value: 0.7938295961252309 and parameters: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.017044113670805362}. Best is trial 0 with value: 0.7938295961252309.
[I 2023-04-13 02:40:58,895] Trial 1 finished with value: 0.7870880428484943 and parameters: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.07707297378664633}. Best is trial 0 with value: 0.7938295961252309.
[I 2023-04-13 02:41:03,350] Trial 2 finished with value: 0.778438740356448 and parameters: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.3349611671245042}. Best is trial 0 with value: 0.7938295961252309.
[I 2023-04-13 02:42:00,168] Trial 3 finished with value: 0.7744016977719167 and parameters: {'solver': 'saga', 'penalty': 'l2', 'C': 0.9199314841297963}. Best is trial 0 with value: 0.7938295961252309.
[I 2023-04-13 02:42:04,826] Trial 4 finished with value: 0.7874539264350459 and parameters: {'s

Наилучшие значения гиперпараметров {'solver': 'saga', 'penalty': 'l2', 'C': 0.017942501310672217}
f1_score на обучающем наборе: 0.79
f1_score на тестовом наборе: 0.78


#### Случайный лес

In [32]:
def optuna_rfc(trial_optuna_rfc):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial_optuna_rfc.suggest_int('n_estimators', 60, 240, 2)
  max_depth = trial_optuna_rfc.suggest_int('max_depth', 4, 30, 1)
  min_samples_leaf = trial_optuna_rfc.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

# cоздаем объект исследования
study_rfc = optuna.create_study(study_name='RandomForestClassifier', direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rfc.optimize(optuna_rfc, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_rfc.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study_rfc.best_value))\
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_rfc.best_params,random_state=42)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-04-13 02:46:14,337] A new study created in memory with name: RandomForestClassifier
[I 2023-04-13 02:46:18,263] Trial 0 finished with value: 0.8095429395195094 and parameters: {'n_estimators': 160, 'max_depth': 11, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8095429395195094.
[I 2023-04-13 02:46:23,219] Trial 1 finished with value: 0.8174083521754032 and parameters: {'n_estimators': 154, 'max_depth': 22, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.8174083521754032.
[I 2023-04-13 02:46:28,971] Trial 2 finished with value: 0.8106278177954387 and parameters: {'n_estimators': 218, 'max_depth': 22, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8174083521754032.
[I 2023-04-13 02:46:31,647] Trial 3 finished with value: 0.8055026882035975 and parameters: {'n_estimators': 94, 'max_depth': 18, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.8174083521754032.
[I 2023-04-13 02:46:34,729] Trial 4 finished with value: 0.805340000803138 and parameters: {'n_es

Наилучшие значения гиперпараметров {'n_estimators': 126, 'max_depth': 29, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.82
f1_score на тестовом наборе: 0.80
